In [1]:
import pandas as pd
import numpy as np

## (a) Select a 2-class dataset with binary nD features. This dataset needs to be derived from text documents.

In [2]:
def rf(fn):
    data = pd.read_csv(fn, sep=' ', header=None, skipinitialspace=True,)
    mdata = data[3:]
    print mdata.shape
    return mdata

data = rf('ad.data')
print 'Data size is ',data.shape
print data.head()


(3276, 1559)
Data size is  (3276, 1559)
   0     1       2     3     4     5     6     7     8     9     ...   1549  \
3    60   468  7.8000     1     0     0     0     0     0     0  ...      0   
4    60   468  7.8000     1     0     0     0     0     0     0  ...      0   
5    60   468  7.8000     1     0     0     0     0     0     0  ...      0   
6    59   460  7.7966     1     0     0     0     0     0     0  ...      0   
7    60   234  3.9000     1     0     0     0     0     0     0  ...      0   

   1550  1551  1552  1553  1554  1555  1556  1557  1558  
3     0     0     0     0     0     0     0     0     1  
4     0     0     0     0     0     0     0     0     1  
5     0     0     0     0     0     0     0     0     1  
6     0     0     0     0     0     0     0     0     1  
7     0     0     0     0     0     0     0     0     1  

[5 rows x 1559 columns]


## (b) Estimate the model parameters and compute a discriminant function based on the distribution in each class. Make sure to use the Naive Bayes assumption.

## (c) Classify the examples and measure your error. Compute the confusion matrix, precision, recall, F-measure, and accuracy.'''


In [3]:
# coding=utf-8
import pandas as pd
import numpy as np


# K fold
def k_fold(size, k, shuffle = False):
    index = np.arange(0,size)
    if shuffle:
        np.random.shuffle(index)
    index = np.reshape(index, (k,size/k))
#     print index
    test = []
    train = []
    for i in range(0,k):
        test.append(index[i])
        temp = np.delete(index,i,0)
        train.append(temp.flatten())
    return test, train

def getParam(train_data):
    xnum = train_data.shape[0]
    fnum = train_data.shape[1]-1
    x = train_data.loc[:,:fnum-1]  #.loc[:,:3]
    print 'datashape', train_data.shape
    print 'xshape',x.shape
    y = train_data[fnum]
    print 'yshape',y.shape
    alfaj = np.zeros((2,fnum))
    alfa = np.zeros(2)
    # sumy = y.sum()
    # alfa[1] = sumy/xnum
    # c0 = y[y == 0]
    # alfa[0] = len(c0)/xnum
    subc1 = train_data[train_data[fnum] == 1]
    subc2 = train_data[train_data[fnum] == 2]
    alfa[0] = len(subc1)
    alfa[1] = len(subc2)
    for i in range(0,fnum):
        alfaj[0][i] = subc1[i].sum()
        alfaj[1][i] = subc2[i].sum()
    # for i in range(0,fnum):
    #     t = y.iloc[i]
    #     alfaj[t-1] += x[i]
    #     alfa[t-1] += 1
    for i in range(0,2):
        alfaj[i] = (alfaj[i]+1)/(alfa[i]+2)
    # print 'alfa shape',alfa.shape
    # print 'alfaj shape', alfaj.shape
    return alfa/xnum, alfaj


def calcm(x,xa,xaj): # x is an example, xa is the alfa of x's class, xaj is also alfaj of that class. so xa is 1*1, xaj is 1*n
    g = 0.0
    l = len(x)
    for i in range(0,l):
        g += x[i]*np.log(xaj[i])+(1-x[i])*np.log(1-xaj[i])
        # print i
    return g+np.log(xa)


def classification(test_data, a, aj):
    xnum = test_data.shape[0]
    fnum = test_data.shape[1]-1
    x_test = test_data.loc[:,:fnum-1]
    # y_test = test_data[fnum]
    prelist = []
    for i in range(0, xnum):
        ex = x_test.iloc[i]
        # print i
        c1 =  calcm(ex,a[0],aj[0])
        c2 =  calcm(ex,a[1],aj[1])
        if c1<c2:
            prelist.append(2)
        else:
            prelist.append(1)
    return prelist

def cMatrix(test_data,prelist):
    fnum = test_data.shape[1]-1
    y_test= test_data[fnum].tolist()
    cm = np.zeros((2,2))
    print len(y_test)
    for i in range(0,len(y_test)):
        rl = y_test[i]
        pre = prelist[i]
        cm[rl-1][pre-1] += 1
    return cm

# Compute the confusion matrix, precision, recall,F-measure, and accuracy
def evalpf(cm):
    tp = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tn = cm[1][1]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    Fm = 2*precision*recall/(precision+recall)
    return (precision,recall,Fm,accuracy)

if __name__ == "__main__":
    data = rf('ad.data')
    total = len(data)
    i_test, i_train = k_fold(total, 6, True)
    prelist = []
    data_test =data.iloc[i_test[0],]
    data_train=data.iloc[i_train[0],]
    alfa, alfaj = getParam(data_train)  #(alfa/xnum, alfaj)
    print 'Alpha:'
    print alfa
    print 'Alphaj:'
    print alfaj

    y_predict = classification(data_test, alfa, alfaj)
    cm = cMatrix(data_test,y_predict)
    print 'confusion matrix is '
    print cm
    result = evalpf(cm)
    print 'result of precision, recall,F-measure, and accuracy are'
    print result





(3276, 1559)
datashape (2730, 1559)
xshape (2730, 1558)
yshape (2730L,)
Alpha:
[ 0.13882784  0.86117216]
Alphaj:
[[  5.74855643e+01   2.72939633e+02   4.72741811e+00 ...,   8.66141732e-02
    4.46194226e-02   7.87401575e-03]
 [  4.46710582e+01   8.63229919e+01   2.47033901e+00 ...,   2.97492563e-03
    5.09987250e-03   1.27496813e-03]]
546
confusion matrix is 
[[  77.    0.]
 [ 469.    0.]]
result of precision, recall,F-measure, and accuracy are
(1.0, 0.14102564102564102, 0.24719101123595508, 0.14102564102564102)
